In [ ]:
# --- Setup ---
!pip install roboflow ultralytics
from ultralytics import YOLO
import os
import yaml
from roboflow import Roboflow
from google.colab import drive
from IPython.display import clear_output

# --- 1. Mount Google Drive ---
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Drive Montado.")

# --- 2. Descargar el Dataset Correcto (v12) ---
#    (Necesario para que el entrenamiento encuentre los datos)
print("\nDownloading dataset v12 from Roboflow...")
rf = Roboflow(api_key="tZR7XZhFvySrbVM3hfH4") # Tu API key
project = rf.workspace("luchini").project("datasetenproceso-irzcf")
version = project.version(12) # La versión usada para el mejor modelo
dataset = version.download("yolov8")
data_yaml_path = f'{dataset.location}/data.yaml'

# --- Corrección de Rutas en data.yaml ---
with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)
data_yaml['train'] = f"{dataset.location}/train/images"
data_yaml['val'] = f"{dataset.location}/valid/images"
if os.path.exists(f"{dataset.location}/test/images"):
    data_yaml['test'] = f"{dataset.location}/test/images"
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

clear_output()
print(f"✅ Dataset v12 descargado y listo en: {dataset.location}")
print(f"   Ruta del data.yaml: {data_yaml_path}")

# --- 3. Definir Path to Your Best Model Weights ---
# *** ¡ASEGÚRATE DE QUE ESTA RUTA ES CORRECTA! ***
best_model_path = '/content/drive/MyDrive/YOLOv8_Training/Coffee_Seg_L-Model_AggressiveAug_v1/weights/best.pt'

if not os.path.exists(best_model_path):
  print(f"❌ Error: Archivo de pesos no encontrado en {best_model_path}")
  raise FileNotFoundError("Verifica la ruta al archivo best.pt de tu mejor modelo.")
else:
  print(f"✅ Pesos del mejor modelo encontrados en: {best_model_path}")

  # --- 4. Cargar el Modelo desde tus mejores pesos ---
  print("\nCargando el modelo para fine-tuning...")
  # *** ¡CAMBIO CLAVE AQUÍ! Cargamos tus pesos entrenados ***
  model = YOLO(best_model_path)
  print("✅ Modelo cargado.")

  # --- 5. Iniciar Fine-Tuning ---
  print("\nIniciando fine-tuning...")
  results = model.train(
     # --- Parámetros Principales ---
     data=data_yaml_path,
     # *** ¡CAMBIO CLAVE! Menos épocas para fine-tuning ***
     epochs=150,          # Reducido (antes 250), la mayor parte del aprendizaje ya está hecha
     patience=40,         # Mantenemos paciencia por si mejora lentamente
     batch=32,
     imgsz=640,
     rect=True,
     optimizer='AdamW',
     # *** ¡CAMBIO CLAVE! Learning Rate reducido ***
     lr0=0.001,           # Reducido 10x (default era 0.01)
     lrf=0.0001,          # Reducido 10x (default era 0.01, mantener proporción o incluso más bajo)

     # --- Parámetros de Guardado ---
     project='/content/drive/MyDrive/YOLOv8_Training',
     # *** ¡NUEVO NOMBRE PARA LA CORRIDA DE FINE-TUNING! ***
     name='Coffee_Seg_L-Model_AggressiveAug_Finetuned',
     exist_ok=True,

     # --- Aumentos "On-the-Fly" (MANTENER LOS MISMOS QUE DIERON EL MEJOR RESULTADO) ---
     degrees=15.0,
     translate=0.1,
     scale=0.5,
     shear=5.0,
     flipud=0.5,
     fliplr=0.5,
     hsv_h=0.015,
     hsv_s=0.7,
     hsv_v=0.4,
     mosaic=1.0,
     mixup=0.0,         # Mantenemos mixup apagado, ya que funcionó mejor sin él
     copy_paste=0.1
  )

  print("\n✅ Fine-tuning completado.")

✅ Dataset v12 descargado y listo en: /content/DatasetEnProceso-12
   Ruta del data.yaml: /content/DatasetEnProceso-12/data.yaml
✅ Pesos del mejor modelo encontrados en: /content/drive/MyDrive/YOLOv8_Training/Coffee_Seg_L-Model_AggressiveAug_v1/weights/best.pt

Cargando el modelo para fine-tuning...
✅ Modelo cargado.

Iniciando fine-tuning...
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/DatasetEnProceso-12/data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_